<a href="https://colab.research.google.com/github/saiashirwad/relation-prediction-2/blob/master/Relation_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/My Drive/Thesis/code/relation-prediction-2

/content/drive/My Drive/Thesis/code/relation-prediction-2


In [0]:
%load_ext autoreload
%autoreload 

In [0]:
import matplotlib.pyplot as plt


In [0]:
pip install torchkge --quiet

In [0]:
pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html --quiet

In [0]:
from tqdm import * 
import torch 
import torch.nn as nn 
from functools import reduce 
from operator import mul 
from torch_scatter import scatter 
from sklearn.metrics.pairwise import pairwise_distances 

In [0]:
from typing import List

In [0]:
from train import *

In [0]:
kg_train, kg_test, kg_val = load_fb15k237()

In [0]:
args = Args(100, 200, 100, 2, 100, 40000, 0.001, 10, 'cuda', 'sgd')

In [11]:
n_ent, n_rel = kg_train.n_ent, kg_train.n_rel
total_triplets = get_valid_triplets(kg_train, kg_test, kg_val)

Number of unique triplets: 620232


In [0]:
def loss_transe(triplets, neg_sampling_ratio, ent_embed, rel_embed, device='cpu'):
    """
    Triplets order: src, dst, rel
    """
    n = len(triplets)
    if type(triplets) == np.ndarray:
        triplets = torch.from_numpy(triplets)

    pos_triplets = triplets[:n // (neg_sampling_ratio + 1)]
    pos_triplets = torch.cat([pos_triplets for _ in range(neg_sampling_ratio)])

    neg_triplets = triplets[n // (neg_sampling_ratio + 1):]


    src_embed_ = ent_embed[pos_triplets[:, 0]]
    dst_embed_ = ent_embed[pos_triplets[:, 1]]
    rel_embed_ = rel_embed[pos_triplets[:, 2]]

    x = src_embed_ + rel_embed_ - dst_embed_
    pos_norm = torch.norm(x, p=2, dim=1)


    src_embed_ = ent_embed[neg_triplets[:, 0]]
    dst_embed_ = ent_embed[neg_triplets[:, 1]]
    rel_embed_ = rel_embed[neg_triplets[:, 2]]

    x = src_embed_ + rel_embed_ - dst_embed_
    neg_norm = torch.norm(x, p=2, dim=1)

    y = torch.ones(len(pos_triplets)).to(device)

    loss_fn = nn.MarginRankingLoss(margin=5)
    loss = loss_fn(pos_norm, neg_norm, y)

    return loss


In [0]:
dataloader = DataLoader(kg_train, batch_size=args.batch_size, shuffle=False, pin_memory=cuda.is_available())
ent_embed, rel_embed = get_init_embed()

In [0]:
batches = [b for b in dataloader]

In [0]:
class SNAFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, edge, edge_w, N, E, out_features):
        a = torch.sparse_coo_tensor(
            edge, edge_w, torch.Size([N, N, out_features]))
        b = torch.sparse.sum(a, dim=1)
        ctx.N = b.shape[0]
        ctx.outfeat = b.shape[1]
        ctx.E = E
        ctx.indices = a._indices()[0, :]

        return b.to_dense()

    @staticmethod
    def backward(ctx, grad_output):
        grad_values = None
        if ctx.needs_input_grad[1]:
            edge_sources = ctx.indices

            if(torch.cuda.is_available()):
                edge_sources = edge_sources.cuda()

            grad_values = grad_output[edge_sources]
        return None, grad_values, None, None, None

class SparseNeighborhoodAggregation(nn.Module):
    def forward(self, edge, edge_w, N, E, out_features):
        return SNAFunction.apply(edge, edge_w, N, E, out_features)


![](https://i.imgur.com/2v9bxG1.png)

In [0]:
class KGLayer(nn.Module):
    def __init__(self, n_entities, n_relations, ent_embed, rel_embed, in_dim, out_dim, input_drop=0.5, concat=True, device="cuda"):
        super(KGLayer, self).__init__()

        self.n_entities = n_entities
        self.n_relations = n_relations
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.device = device

        self.a = nn.Linear(3 * in_dim, out_dim).to(device)
        nn.init.xavier_normal_(self.a.weight.data, gain=1.414)

        self.concat = concat

        self.a_2 = nn.Linear(out_dim, 1).to(device)
        nn.init.xavier_normal_(self.a_2.weight.data, gain=1.414)

        self.sparse_neighborhood_aggregation = SparseNeighborhoodAggregation()

        self.ent_embed = nn.Embedding(n_entities, in_dim, max_norm=1, norm_type=2).to(device)
        self.rel_embed = nn.Embedding(n_relations, in_dim, max_norm=1, norm_type=2).to(device)
        
        nn.init.xavier_normal_(self.ent_embed.weight.data, 1.414)
        nn.init.xavier_normal_(self.rel_embed.weight.data, 1.414)
        
        # self.ent_embed.weight = nn.Parameter(ent_embed.to("cuda"))
        # self.rel_embed.weight = nn.Parameter(rel_embed.to("cuda"))

        self.input_drop = nn.Dropout(input_drop)

        self.bn0 = nn.BatchNorm1d(3 * in_dim).to(device)
        self.bn1 = nn.BatchNorm1d(out_dim).to(device)

    
    def forward(self, triplets):

        N = self.n_entities

        h = torch.cat((
            self.ent_embed(triplets[:, 0]),
            self.ent_embed(triplets[:, 1]),
            self.rel_embed(triplets[:, 2])
        ), dim=1)
        h_ = torch.cat((
            self.ent_embed(triplets[:, 1]),
            self.ent_embed(triplets[:, 0]),
           -self.rel_embed(triplets[:, 2])  
        ), dim=1)
        h = torch.cat((h, h_)) # should I check for the presence of t,h,r in triplets?

        h = self.input_drop(self.bn0(h))
        c = self.bn1(self.a(h))
        b = -F.leaky_relu(self.a_2(c))
        e_b = torch.exp(b)

        temp = triplets.t()
        edges = torch.stack((
            torch.cat([temp[0], temp[1]]),
            torch.cat([temp[1], temp[0]])
        ))

        ebs = self.sparse_neighborhood_aggregation(edges, e_b, N, e_b.shape[0], 1)
        temp1 = e_b * c

        hs = self.sparse_neighborhood_aggregation(edges, temp1,  N, e_b.shape[0], self.out_dim)

        ebs[ebs == 0] = 1e-12
        h_ent = hs / ebs

        index = triplets[:, 2]
        h_rel  = scatter(temp1[ : temp1.shape[0]//2, :], index=index, dim=0, reduce="mean")
        h_rel_ = scatter(temp1[temp1.shape[0]//2 : , :], index=index, dim=0, reduce="mean")  

        h_rel = h_rel - h_rel_  

        if self.concat:
            return F.elu(h_ent), F.elu(h_rel)
        else:
            return h_ent, h_rel

In [0]:
model = KGLayer(n_ent, n_rel, ent_embed, rel_embed, 100, 100, 0.5, True, "cuda")

In [0]:
optimizer = SGD(model.parameters(), lr=0.001)

In [19]:
for epoch in tnrange(5):
    losses = []
    for i in range(len(batches)):
        batch = batches[i]
        triplets = torch.stack(batch)
        triplets, labels, nodes, edges = negative_sampling(triplets, n_ent, args.negative_rate)
        triplets, labels = triplets.to(args.device), labels.to(args.device)
    
        model.zero_grad()
    
        # start = time.time()
        model.train()
        ent_embed_, rel_embed_ = model(triplets)
        loss = loss_transe(triplets, args.negative_rate, ent_embed_, rel_embed_, device="cuda")
        loss.backward()
        optimizer.step()
    
        torch.cuda.empty_cache()
    
        losses.append(loss.item())
        torch.cuda.empty_cache()
    
    print(f"epoch: {epoch}, loss: {sum(losses) / len(losses)}")
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """Entry point for launching an IPython kernel.


epoch: 0, loss: 4.533815588269915
epoch: 1, loss: 4.530610629490444
epoch: 2, loss: 4.5276216098240445
epoch: 3, loss: 4.524810041700091
epoch: 4, loss: 4.522953987121582



In [0]:
torch.save(model.state_dict(), "model.save")

In [21]:
model.load_state_dict(torch.load("model.save"))

<All keys matched successfully>

## Evaluation

In [0]:
def generate_eval_triplets(triplet: List[torch.Tensor], pos="head", n_ent=14541):
    if pos == "head":
        triplet = torch.tensor([triplet[1].item(), triplet[2].item()]).to(torch.long)
        triplets = triplet.repeat(n_ent).view(-1, 2).t()
        triplets = torch.stack((
            torch.arange(n_ent),
            triplets[0], 
            triplets[1]
        ))
    elif pos == "tail":
        triplet = torch.tensor([triplet[0].item(), triplet[2].item()]).to(torch.long)
        triplets = triplet.repeat(n_ent).view(-1, 2).t()
        triplets = torch.stack((
            triplets[0], 
            torch.arange(n_ent),
            triplets[1]
        ))
    
    return triplets.t()

In [0]:
dl = DataLoader(kg_val, 1, shuffle=True)
data = [d for d in dl]
triplets = data[0]

In [28]:
# n = len(data)
n = 1000
n_dim = 100
model.eval()
with torch.no_grad():
    for i in tnrange(n):
        # # head
        triplets_h = generate_eval_triplets(data[i], "head", n_ent)
        triplets_h, _, _, _ = negative_sampling(triplets_h, n_ent, 0)
        triplets_h = triplets_h.to("cuda")
        ee, re = model(triplets_h)
    
        dst = ee[data[i][1]].squeeze()
        rel = re[data[i][2]].squeeze()
        dist = ee + (rel - dst).repeat(n_ent).view(-1, 100)
        head_preds = torch.topk(torch.norm(dist, dim=1), k=n_ent).indices.cpu().tolist()
    
        # # # # tail
        triplets_t = generate_eval_triplets(data[i], "tail", n_ent)
        triplets_t, _, _, _ = negative_sampling(triplets_t, n_ent, 0)
        triplets_t = triplets_t.to("cuda")
        ee, re = model(triplets_t)
    
        src = ee[data[i][0]].squeeze()
        rel = re[data[i][2]].squeeze()
        dist = (src + rel).repeat(n_ent).view(-1, 100) - ee
        tail_preds = torch.topk(torch.norm(dist, dim=1), k=n_ent).indices.cpu().tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """


In [25]:
triplets_t

tensor([[14097,     0,   178],
        [14097,     1,   178],
        [14097,     2,   178],
        ...,
        [14097, 14538,   178],
        [14097, 14539,   178],
        [14097, 14540,   178]], device='cuda:0')

In [27]:
triplets_h

NameError: ignored

In [0]:
torch.arange(20).cuda()

In [0]:
for i in range(1):
    print(8)